<a href="https://colab.research.google.com/github/victoreeckh/Model_free_HCA_1_feeder_v1/blob/main/PFE_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Power Flow Engine model

In [8]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import time
import os
import json

In [ ]:
!pip install dss_python #own code
!pip install dss

In [4]:
import dss
dss_via_python = dss.DSS
print(dss_via_python) # check object type

### Data acquisition

In [11]:
#import files from FNAData/Network
from google.colab import files
uploaded = files.upload()

Saving 65019_74478_Mod_branches.json to 65019_74478_Mod_branches.json
Saving 65019_74478_Mod_buses.json to 65019_74478_Mod_buses.json
Saving 65019_74478_mod_configuration.json to 65019_74478_mod_configuration.json
Saving 65019_74478_Mod_devices.json to 65019_74478_Mod_devices.json


In [12]:
#import files from FNAData/ScenariosRobustSolution
from google.colab import files
uploaded = files.upload()

Saving House0.csv to House0.csv
Saving House1.csv to House1.csv
Saving House2.csv to House2.csv
Saving House3.csv to House3.csv
Saving House4.csv to House4.csv
Saving House5.csv to House5.csv
Saving House6.csv to House6.csv
Saving House7.csv to House7.csv
Saving House8.csv to House8.csv
Saving House9.csv to House9.csv
Saving House10.csv to House10.csv
Saving House11.csv to House11.csv
Saving House12.csv to House12.csv
Saving House13.csv to House13.csv
Saving House14.csv to House14.csv
Saving House15.csv to House15.csv
Saving House16.csv to House16.csv
Saving House17.csv to House17.csv
Saving House18.csv to House18.csv
Saving House19.csv to House19.csv
Saving House20.csv to House20.csv
Saving House21.csv to House21.csv
Saving House22.csv to House22.csv
Saving House23.csv to House23.csv
Saving House24.csv to House24.csv
Saving House25.csv to House25.csv
Saving House26.csv to House26.csv
Saving House27.csv to House27.csv
Saving House28.csv to House28.csv
Saving House29.csv to House29.csv


In [31]:
House0 = pd.read_csv('/content/House0.csv', header=None)
print(House0.shape)

(24, 1000)


In [13]:
with open('/content/65019_74478_Mod_branches.json', 'r') as file:
    branches = json.load(file)
with open('/content/65019_74478_Mod_buses.json', 'r') as file:
    buses = json.load(file)
with open('/content/65019_74478_Mod_devices.json', 'r') as file:
    devices = json.load(file)
    LVcustomers = devices['LVcustomers']
    solarGens = devices['solarGens']
with open('/content/65019_74478_mod_configuration.json', 'r') as file:
    configuration = json.load(file)
print(branches)
print(buses)
print(devices)
print(configuration)

[{'branchId': 0, 'cableLength': 17.0, 'cableType': 'BT - RV 0,6/1 KV 3(1*150 KAL) + 1*95 KAL', 'downBusId': 1, 'type': 'Cable', 'upBusId': 0}, {'branchId': 1, 'cableLength': 23.0, 'cableType': 'BT - RV 0,6/1 KV 3(1*150 KAL) + 1*95 KAL', 'downBusId': 2, 'type': 'Cable', 'upBusId': 1}, {'branchId': 2, 'cableLength': 5.0, 'cableType': 'BT - RV 0,6/1 KV 4*95 KAL', 'downBusId': 3, 'type': 'Cable', 'upBusId': 2}, {'branchId': 3, 'cableLength': 10, 'cableType': 'BT - RV 0,6/1 KV 4*95 KAL', 'downBusId': 4, 'type': 'Cable', 'upBusId': 3}, {'branchId': 4, 'cableLength': 1.0, 'cableType': 'BT - RV 0,6/1 KV 3(1*150 KAL) + 1*95 KAL', 'downBusId': 5, 'type': 'Cable', 'upBusId': 2}, {'branchId': 5, 'cableLength': 25.0, 'cableType': 'BT - RZ 0,6/1 KV 3*150 AL/95 ALM', 'downBusId': 6, 'type': 'Cable', 'upBusId': 5}, {'branchId': 6, 'cableLength': 20.0, 'cableType': 'BT - RZ 0,6/1 KV 3*150 AL/95 ALM', 'downBusId': 7, 'type': 'Cable', 'upBusId': 6}, {'branchId': 7, 'cableLength': 2.0, 'cableType': 'BT - 

In [14]:
impedance_dict = {
    "BT - Desconocido BT": [0.21, 0.075],
    "BT - MANGUERA": [0.3586, 0.089],
    "BT - RV 0,6/1 KV 2*16 KAL": [2.14, 0.09],
    "BT - RV 0,6/1 KV 2*25 KAL": [1.34, 0.097],
    "BT - RV 0,6/1 KV 3(1*150 KAL) + 1*95 KAL": [0.2309, 0.085],
    "BT - RV 0,6/1 KV 3(1*240 KAL) + 1*150 KAL": [0.1602, 0.079],
    "BT - RV 0,6/1 KV 3(1*240 KAL) + 1*95 KAL": [0.1602, 0.079],
    "BT - RV 0,6/1 KV 4*25 KAL": [1.34, 0.097],
    "BT - RV 0,6/1 KV 4*50 KAL": [0.71849, 0.093],
    "BT - RV 0,6/1 KV 4*95 KAL": [0.3586, 0.089],
    "BT - RX 0,6/1 KV 2*16 Cu": [1.23, 0.08],
    "BT - RX 0,6/1 KV 2*2 Cu": [9.9, 0.075],
    "BT - RX 0,6/1 KV 2*4 Cu": [4.95, 0.075],
    "BT - RX 0,6/1 KV 2*6 Cu": [3.3, 0.075],
    "BT - RZ 0,6/1 KV 2*16 AL": [2.14, 0.09],
    "BT - RZ 0,6/1 KV 3*150 AL/80 ALM": [0.2309, 0.85],
    "BT - RZ 0,6/1 KV 3*150 AL/95 ALM": [0.2309, 0.85],
    "BT - RZ 0,6/1 KV 3*25 AL/54,6 ALM": [1.34, 0.097],
    "BT - RZ 0,6/1 KV 3*35 AL/54,6 ALM": [0.9073, 0.095],
    "BT - RZ 0,6/1 KV 3*50 AL/54,6 ALM": [0.718497, 0.093],
    "BT - RZ 0,6/1 KV 3*70 ALM/54,6 AL": [0.4539, 0.091],
    "BT - RZ 0,6/1 KV 3*95 AL/54,6 ALM": [0.3586, 0.089],
    "BT - RZ 0,6/1 KV 4*16 AL": [2.14, 0.09],
    "aansluitkabel": [1.15, 0.150]
}

currentmax_dict = {
    "BT - Desconocido BT": 200,
    "BT - MANGUERA": 150,  # 200 certain 40.18 # 150
    "BT - RV 0,6/1 KV 2*16 KAL": 75,
    "BT - RV 0,6/1 KV 2*25 KAL": 100,
    "BT - RV 0,6/1 KV 3(1*150 KAL) + 1*95 KAL": 264,
    "BT - RV 0,6/1 KV 3(1*240 KAL) + 1*150 KAL": 344,
    "BT - RV 0,6/1 KV 3(1*240 KAL) + 1*95 KAL": 344,
    "BT - RV 0,6/1 KV 4*25 KAL": 100,
    "BT - RV 0,6/1 KV 4*50 KAL": 150,
    "BT - RV 0,6/1 KV 4*95 KAL": 230,
    "BT - RX 0,6/1 KV 2*16 Cu": 95,
    "BT - RX 0,6/1 KV 2*2 Cu": 30,
    "BT - RX 0,6/1 KV 2*4 Cu": 40,
    "BT - RX 0,6/1 KV 2*6 Cu": 50,
    "BT - RZ 0,6/1 KV 2*16 AL": 75,
    "BT - RZ 0,6/1 KV 3*150 AL/80 ALM": 264,
    "BT - RZ 0,6/1 KV 3*150 AL/95 ALM": 264,
    "BT - RZ 0,6/1 KV 3*25 AL/54,6 ALM": 78.98,
    "BT - RZ 0,6/1 KV 3*35 AL/54,6 ALM": 120,
    "BT - RZ 0,6/1 KV 3*50 AL/54,6 ALM": 118.47,
    "BT - RZ 0,6/1 KV 3*70 ALM/54,6 AL": 160,
    "BT - RZ 0,6/1 KV 3*95 AL/54,6 ALM": 182.21,
    "BT - RZ 0,6/1 KV 4*16 AL": 75,
    "aansluitkabel": 200
}


In [15]:
print(len(branches))
print(len(buses))
print(len(devices))
print(len(LVcustomers))
print(len(solarGens))

75
76
2
52
0


### Build LV network

In [16]:
#Set up dss engine
dss_engine = dss.DSS
DSSText = dss_engine.Text
DSSCircuit = dss_engine.ActiveCircuit
DSSSolution = dss_engine.ActiveCircuit.Solution
ControlQueue = dss_engine.ActiveCircuit.CtrlQueue
dss_engine.AllowForms = 0

In [26]:
#initial commands
DSSText.Command = 'Clear'
DSSText.Command = 'New Circuit.Simple_LV_Network basekv=22.0 pu=1.00 angle=0 phases=1'
DSSText.Command = 'Set DefaultBaseFrequency=50'

In [ ]:
#Transformer set-up?

In [27]:
#Set up branches

for branch in branches:
  branchId = branch["branchId"]
  cableLength = branch["cableLength"]
  cableType = branch["cableType"]
  R1 = impedance_dict[cableType][0]
  X1 = R1 = impedance_dict[cableType][1]
  downBusId = branch["downBusId"]
  upBusId = branch["upBusId"]

  cmd = f'new line.L{upBusId}{downBusId} bus1={upBusId} bus2={downBusId} length={cableLength} phases=1 units=km R1={R1} X1={X1}'
  DSSText.Command = cmd

In [28]:
#Set up loads

for load in LVcustomers:
  busId = load["busId"]
  deviceId = load['deviceId']

  cmd = f'new load.Load_Bus{busId}_Dev{deviceId} bus1={busId} phases=1 kV=0.230 pf=0.95 model=1 conn=wye Vminpu=0.9 Vmaxpu=1.10 status=variable'
  DSSText.Command = cmd